**Remark:** This notebook is based on materials from the Gurobi Optimization Community notebooks.

# Resource Assignment Problem formulation

Consider three job positions: Tester, Java-Developer, and Architect.

Consider three resources: Carlos, Joe, and Monika.

## Data 

The ability to perform each of the jobs by each of the resources is illustrated by the following matching scores table:

![Resource Allocation Problem Data Image](assets/rap_data.png)


**Assumption**: Only one resource can be assigned to a job, and only one job can be assigned to a resource.

## Problem statement

Determine an assignment that ensures that each job is fulfilled and each resource is assigned to at most one job in order to maximize the total matching scores of the assignments.

## Decision variables

The decision variable $x_{r,\; j} = 1$ represents that resource r is assigned to job j, and 0 otherwise, for  r=1,2,3 and 𝑗=1,2,3.

## Constraints

### Jobs constraints

For each job 𝑗=1,2,3, exactly one resource from r=1,2,3 must be assigned.

Constraint (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

Constraint (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

Constraint (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### Resources constraints

For each resource = r=1,2,3, at most one job from r=1,2,3 can be assigned.

Constraint (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## Objective function

The objective function is to maximize the total matching score of the assignments while satisfying the jobs and resources constraints.

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




First, let's install gurobipy as needed

In [4]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [5]:
# import gurobi library
import gurobipy as gp
from gurobipy import *

## Data
The list R contains the names of the three resources: Carlos, Joe, and Monika. 

The list J contains the names of the job positions: tester, java-developer, and architect.

**Math notation**

$r \in R$ means that a resource with index r is in the set (list) R.

$j \in J$ means that a job with index j is in the set (list) J.

In [7]:
# resources and jobs sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

The following “multidict” function describes the matching score associated with each possible combination of a resource and job.

**Math notation**

Let $ms_{r,\;j}$ be the matching score of resource  $r \in R$  with respect to job  $j \in J$.

Let $C_{r,\;j}$ be the cost of assigning resource  $r \in R$  to job  $j \in J$.

Let $B$ be the budget available.

In [9]:
# matching score data
combinations, ms, C = multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# Budget available
#B = 6
B=5

The following function generates an empty model object “m” and takes the string “RAP” model name as its argument.

In [11]:
# Declare and initialize model
m = Model('RAP')

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22


## Decision variables

The decision variable $x_{r,\; j} = 1$ represents that resource r is assigned to job j, and 0 otherwise, for  r=1,2,3 and 𝑗=1,2,3.

The “addVars()” method defines the decision variables of the model object “m”.  

**Math notation**

Let $x_{r,\; j} = 1$ if resource $r \in R$  is assigend to job $j \in J$, and zero otherwise.

Let $g_{j} = 1$ if job $j \in J$ cannot be filled, and zero otherwise. This variable is a gap variable that indicates that a job cannot be filled.


In [13]:
# Create decision variables for the RAP model
#x = m.addVars(combinations, name="assign")
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Create gap variables for the RAP model
g = m.addVars(J, name="gap")

## Jobs constraints

For each job 𝑗=1,2,3, exactly one resource from r=1,2,3 must be assigned.

Constraint (Tester=1): $x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} + g_{1} = 1$

Constraint (Java-Developer=2): $x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} + g_{2} = 1$

Constraint (Architect=3): $x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} + g_{3} = 1$

The “addConstrs()” method defines the constraints of the model object “m”. 

**Math notation**

For each job $j \in J$, exactly one resouce must be assigned:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1 
$$



In [15]:
# create jobs  constraints
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), 'job')

## Resources constraints

For each resource = r=1,2,3, at most one job from r=1,2,3 can be assigned.

Constraint (Carlos=1): $x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

Constraint (Joe=2): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

Constraint (Monika=3): $x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

The “addConstrs()” method defines the constraints of the model object “m”. 

**Math notation**

For each resource $r \in R$, at most one job can be assigned:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1 
$$

In [17]:
# create resources constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## Budget constraint

The total cost of assigning resources to jobs should be less or equal than the budget available.

$$
\sum_{r \; \in \; R} \sum_{j \; \in \; J} C_{r, j}x_{r, j} \leq B
$$

In [19]:
budget = m.addConstr((x.prod(C) <= B), 'budget')

## Objective Function

The objective function is to maximize the total matching score of the assignments.

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

The “setObjective()” method defines the objective function of the model object “m”. 

**Math notation**

Notice that 
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

Hence, the objective function can be expressed as follows

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j} -BigM \sum_{j \in J} g_{j}
$$


In [21]:
# Penalty for not filling a job position
BIGM =101

# The objective is to maximize total matching score of the assignments
m.setObjective(x.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)

In [22]:
# save model for inspection
m.write('models/RAP3.lp')

In [23]:
# run optimization engine
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.000

In [24]:
# display optimal values of decision variables
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# display optimal total matching score
print('Optimal objective function value', m.objVal)   

# Compute total matching score from assignment  variables
total_matching_score = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_matching_score = total_matching_score + ms[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  


assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
Optimal objective function value 52.0
Total matching score:  153.0


### Sample a Random Scenario

In [26]:
import names
import random
import numpy as np
from gurobipy import *
from itertools import product

def generate_scenario(num_resources=200, num_jobs=200, roles=None,
                      score_mu=50, score_sigma=15, seed=10101):
    random.seed(seed)
    np.random.seed(seed)
    if roles is None:
        roles = {"Architect", "BackEndEngineer", "FrontEndEngineer",
                     "Tester", "DataScientist", "DataEngineer"}
    # P.D.F. of resource costs follows Benford's law, having support {1,2,...,9}
    benford = [np.log10((i+1)/i) for i in range(1,10)]
    # Sample resource names
    resources = {names.get_full_name() for i in range(num_resources)}
    # Sample job requirements, given that all roles are equally likely to be selected
    req = np.random.multinomial(num_jobs, [1/len(roles)]*len(roles), size=1)[0]
    jobs = set()
    # Assign ID to each job position
    for i, role in enumerate(roles):
        jobs = jobs.union(set(map(''.join, zip([role]*req[i], [str(x).zfill(int(np.log10(num_jobs))+1) for x in range(1,req[i]+1)]))))
    scores = {}
    costs = {}
    # Sample matching score and cost for each potential assignment
    for pair in product(resources, jobs):
        scores[pair] = int(np.clip(np.random.normal(score_mu, score_sigma), 0, 100))
        costs[pair] = random.choices(list(range(1,10)), weights=benford, k=1)[0]
    return resources, jobs, scores, costs 

In [27]:
res, job, ms, cst = generate_scenario(seed=11111)
budget = 200

### Get a Greedy Solution

In [29]:
def greedy_solve(resources, jobs, scores, costs, budget):
    assign = set()
    total_score = 0
    remaining_budget = budget
    while remaining_budget > 0 and len(scores.keys()) > 0:
        selection = max(scores, key=scores.get)
        assign.add(selection)
        total_score += scores[selection]
        remaining_budget -= costs[selection]
        # Remove potential assignments related to the resource/job of new selection
        res_filter = list(filter(lambda x: x[0] == selection[0], scores))
        job_filter = list(filter(lambda x: x[1] == selection[1], scores))
        blacklist = res_filter + job_filter
        scores = {key: val for key,val in scores.items()
                  if key not in blacklist
                  and costs[key] <= remaining_budget}
    print("Number of assignments: {0}".format(len(assign)))
    print("Total matching score: {0}".format(total_score))
    print("Budget consumed: {0}".format(budget - remaining_budget))
    
    kpi = {}
    kpi["n_assign"] = len(assign)
    kpi["total_ms"] = total_score
    kpi["budget_used"] = budget - remaining_budget
    return assign, kpi
        

In [30]:
greedy_sol, kpi = greedy_solve(res, job, ms, cst, budget)

# Greedy heuristic KPI's 
Greedy_assign = kpi["n_assign"]
Greedy_ms = kpi["total_ms"]

#print('Greedy number assignments: ', Greedy_assign)
#print('Greedy total matching score: ',Greedy_ms)

Number of assignments: 58
Total matching score: 5589
Budget consumed: 200


### Get Optimal Solution

In [32]:
m = Model("RAP")
assign = m.addVars(ms.keys(), vtype=GRB.BINARY, name="assign")
g = m.addVars(job, name="gap")
m.addConstrs((assign.sum("*", j) + g[j]  == 1 for j in job), name="demand")
m.addConstrs((assign.sum(r, "*") <= 1 for r in res), name="supply")
m.addConstr(assign.prod(cst) <= budget, name="Budget")
m.setObjective(assign.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 401 rows, 40200 columns and 120200 nonzeros
Model fingerprint: 0xcca97cf6
Variable types: 200 continuous, 40000 integer (40000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -20200.00000
Presolve time: 0.20s
Presolved: 401 rows, 40200 columns, 120200 nonzeros
Variable types: 0 continuous, 40200 integer (40200 binary)

Root relaxation: objective 1.627500e+04, 592 iterations, 0.07 seconds (0.09 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   

In [33]:
def print_solution(model):
    i = 1
    total_ms = 0
    for var in model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}) {1}: {2}".format(i, var.varName, var.x))
            i += 1
            if "assign" in var.varName:
                total_ms += var.Obj
    print('Total matching score: {0}'.format(total_ms))
    print('Optimal objective function value: {0}'.format(model.objVal))
    return None

# display optimal values of decision variables
print_solution(m)

1) assign[Luis Tamondong,DataEngineer016]: 1.0
2) assign[Robert Douglas,DataScientist016]: 1.0
3) assign[Philip Mcalary,DataScientist039]: 1.0
4) assign[Patrick Rogers,Architect011]: 1.0
5) assign[Heather Lewis,FrontEndEngineer025]: 1.0
6) assign[Paul Lowe,FrontEndEngineer042]: 1.0
7) assign[Carmella Sixon,DataScientist008]: 1.0
8) assign[John Nester,DataScientist028]: 1.0
9) assign[Karen Horton,Tester019]: 1.0
10) assign[Jordan Ward,Tester027]: 1.0
11) assign[Jennifer Hogan,FrontEndEngineer021]: 1.0
12) assign[Crystal Douglas,Tester001]: 1.0
13) assign[Michelle Garnand,BackEndEngineer009]: 1.0
14) assign[Christine Nichols,FrontEndEngineer018]: 1.0
15) assign[Joyce Carrier,Architect026]: 1.0
16) assign[Shirley Begley,DataEngineer001]: 1.0
17) assign[Charles Gomez,FrontEndEngineer041]: 1.0
18) assign[Nicholas Beltrame,DataEngineer029]: 1.0
19) assign[Cordelia Kaylor,Tester011]: 1.0
20) assign[Joey Sampson,Architect029]: 1.0
21) assign[Gladys Worrell,FrontEndEngineer026]: 1.0
22) assign[

In [34]:
# comparing KPI's of greedy heuristic and Gurobi Optimizer
Gurobi_assign = 0
Gurobi_ms = 0
for [r,j] in ms.keys():
    if (abs(assign[r, j].x) > 1e-6):
        Gurobi_assign = Gurobi_assign + assign[r, j].x
        Gurobi_ms = Gurobi_ms + ms[r, j]*assign[r, j].x
        
#print('Gurobi total assignments: ', Gurobi_assign)
#print('Gurobi total matching score: ', Gurobi_ms)

#Ratio of Greedy assignments respect to Gurobi assignments
assign_ratio = 100*Greedy_assign/Gurobi_assign
ms_ratio = 100*Greedy_ms/Gurobi_ms

print('Assignment ratio: ',"%.2f" % assign_ratio, '%')
print('Matching score ratio: ',"%.2f" % ms_ratio, '%')

Assignment ratio:  29.00 %
Matching score ratio:  34.34 %
